# Download street network

Download street network data from Overture Maps.

In [4]:
from core.generate_streets import read_overture_region_streets

import glob

import geopandas as gpd
import datetime

In [ ]:
regions_v = "2"

## Run sequentially for all streets

In [6]:
streets_dir = "/data/uscuni-ulce/overture_streets/"
regions_datadir = "/data/uscuni-ulce/"

In [7]:
region_hulls = gpd.read_parquet(
    regions_datadir + "regions/" + f"region_hulls_v{regions_v}.parquet"
)
region_hulls.shape

(856, 1)

## Run in parallel for all non-procesed streets in region_hulls

In [8]:
def process_and_save(region_hull, region_id):
    print("Processing", region_id, datetime.datetime.now())
    streets = read_overture_region_streets(region_hull, region_id)
    streets.to_parquet(streets_dir + f"streets_{region_id}.pq")

In [9]:
downloaded_streets = glob.glob(streets_dir + "*.pq")
downloaded_streets = [int(s.split("_")[-1][:-3]) for s in downloaded_streets]

In [10]:
region_hulls = region_hulls.loc[~region_hulls.index.isin(downloaded_streets),]
region_hulls

,convex_hull
labels,
153512,"POLYGON ((3960400 2975800, 3953200 2976800, 39..."
153650,"POLYGON ((3866100 3061900, 3860400 3062200, 38..."
153752,"POLYGON ((3863100 3136800, 3861000 3140800, 38..."
153804,"POLYGON ((3871800 3135900, 3870200 3136600, 38..."
154059,"POLYGON ((3908500 2998500, 3905000 2999000, 39..."
154947,"POLYGON ((3976100 3073900, 3975200 3074300, 39..."
155610,"POLYGON ((4008200 3082200, 4007700 3082300, 40..."
155668,"POLYGON ((3992700 2941700, 3963300 2974900, 39..."
155971,"POLYGON ((4044900 3009300, 4030900 3009600, 40..."


In [11]:
%%capture cap

from joblib import Parallel, delayed

n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(process_and_save)(region_hull.iloc[0], region_id)
    for region_id, region_hull in region_hulls.to_crs(epsg=4326).iterrows()
)

Processing 154947 2025-05-22 11:23:29.469840
Processing 153752 2025-05-22 11:23:29.440707
Processing 153804 2025-05-22 11:23:29.440341
Processing 155610 2025-05-22 11:23:29.455326
Processing 154059 2025-05-22 11:23:29.464036
Processing 155668 2025-05-22 11:23:29.450969
Processing 155971 2025-05-22 11:23:29.478108
Processing 153650 2025-05-22 11:23:29.729198


In [12]:
gdf = gpd.read_parquet(streets_dir + f"streets_{153804}.pq")

In [ ]:
gdf.explore()

Processing 153512 2025-05-22 11:23:29.496504


In [21]:
cap.stderr

'/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.\n  warnings.warn(\n'